In [1]:
from aicspylibczi import CziFile
import tifffile
from patchify import patchify, unpatchify
from pystackreg import StackReg
from tqdm import tqdm


import matplotlib.pyplot as plt
import numpy as np
from skimage import transform

import time
import os
import cv2
import itertools
import gc
import math

from sys import getsizeof

In [2]:
import psutil
psutil.virtual_memory().total/(1024**3)

63.74656677246094


todo:

read_czi_generate_scale_img:
    generate low res single layer img?
    generate specific scene
    handle really large image file

read_czi_generate_edof_img:
    generate specific scene
    handle really large image file
    progress calculating

split the large contour into multipieces, 
and read stack, edof for each,
and stitch back.
Possible error, edof pad will cause problem when stitch back

estimating the file size


read czi -> multi z layers img -> split into big patch -> each big patch first layer tissue:background, if ratio too small, skip? -> each edof patch in the big patch, tissue image quality, crystal? dust?, delete unwanted? -> EDOF -> full edof image

In [3]:
# cv2.INTER_AREA,
# cv2.INTER_BITS,
# cv2.INTER_BITS2,
# cv2.INTER_CUBIC,
# cv2.INTER_LANCZOS4,
# cv2.INTER_LINEAR,
# cv2.INTER_LINEAR_EXACT,
# cv2.INTER_NEAREST

# Utils

In [4]:
def writeTiffStk(img_stack,save_img_path):
    with tifffile.TiffWriter(save_img_path) as stack:
        for img in img_stack:
            stack.save(img)

# DeprecationWarning: <tifffile.TiffWriter.save> is deprecated. Use TiffWriter.write
#   stack.save(img)

def writeTiff(img_stack,save_img_path):
    tifffile.imwrite(save_img_path, img_stack, photometric='rgb',metadata={'axes':'YXC'})

def readTiffStk(img_path,move_axis=True):
    img = tifffile.imread(img_path)
    if move_axis:
        img = np.moveaxis(img,1,-1)
    return img

In [5]:
def crop_content(image):
    bounding_box = find_minimum_bounding_box(image)
    return extract_content(image, bounding_box)

def find_minimum_bounding_box(image):
    """Find the minimum bounding box for the non-zero content of an image.

    Args:
    image: A 2D numpy array representing the image.

    Returns:
    Tuple of (min_x, min_y, max_x, max_y) that forms the bounding box.
    """
    # Find rows and columns where the image is non-zero
    non_zero_coords = np.argwhere(image)

    # Extract separate arrays for rows and columns
    rows = non_zero_coords[:, 0]
    cols = non_zero_coords[:, 1]

    # Find minimum and maximum values
    min_row, max_row = rows.min(), rows.max()
    min_col, max_col = cols.min(), cols.max()

    # Calculate width and height
    width = max_col - min_col + 1
    height = max_row - min_row + 1

    # Return the bounding box coordinates in (x, y, width, height) format
    return (min_col, min_row, width, height)

# Example usage:
# image = load_your_image_here()
# bounding_box = find_minimum_bounding_box(image)
# print("Bounding box:", bounding_box)

def extract_content(image, bounding_box_xywh):
    """
    Extracts the content from the image based on a bounding box in (x, y, width, height) format.

    Args:
    image: A 2D numpy array representing the image.
    bounding_box_xywh: A tuple of (x, y, width, height) defining the bounding box.

    Returns:
    A 2D numpy array containing only the content within the bounding box.
    """
    x, y, width, height = bounding_box_xywh
    min_x, min_y = x, y
    max_x, max_y = x + width, y + height

    return image[min_y:max_y, min_x:max_x]

# Example usage:
# image = load_your_image_here()
# bounding_box_xywh = (100, 150, 50, 75)  # Example: (x, y, width, height)
# content_image = extract_content(image, bounding_box_xywh)

In [6]:
def apply_padding(res, x_pad, y_pad, x_window_size, y_window_size):
    """Apply padding to the sliced image data.

    Args:
    res: The result from slicing the image data.
    x_pad: The padding to be applied on the x-axis. If None, no padding is applied.
    y_pad: The padding to be applied on the y-axis. If None, no padding is applied.
    x_window_size: The size of the window on the x-axis.
    y_window_size: The size of the window on the y-axis.

    Returns:
    Padded image data.
    """
    x, y = res.shape[1], res.shape[2]
    npad = ((0, 0),
            (0, x_pad if x_pad is not None else 0),
            (0, y_pad if y_pad is not None else 0),
            (0, 0))
    return np.pad(res, pad_width=npad, mode='constant', constant_values=0)

def slice_func(img_data, start_points, window_shape, padding=None):
    """Slice the image data and apply optional padding.

    Args:
    img_data: The original multi-dimensional image data.
    start_points: The starting points for slicing the image data.
    window_shape: The shape of the window for slicing.
    padding: A tuple (x_pad, y_pad) indicating padding on each axis. If None, no padding is applied.

    Returns:
    The sliced (and optionally padded) image data.
    """
    z_window_size, x_window_size, y_window_size, c_window_size = window_shape
    z_start_point, x_start_point, y_start_point, c_start_point = start_points

    res = img_data[
        z_start_point : z_start_point + z_window_size,
        x_start_point : x_start_point + x_window_size,
        y_start_point : y_start_point + y_window_size,
        c_start_point : c_start_point + c_window_size,
    ]

    # Apply padding if necessary
    if padding:
        x_pad, y_pad = padding
        # Adjust padding based on position in the image
        x_pad = x_pad if x_start_point + x_window_size > img_data.shape[1] else None
        y_pad = y_pad if y_start_point + y_window_size > img_data.shape[2] else None
        res = apply_padding(res, x_pad, y_pad, x_window_size, y_window_size)

    return res

In [7]:
def calc_pad_parms(original_image_size, xy_window_size, step_size):
    n_step = 0

    while (n_step-1)*step_size + xy_window_size < original_image_size:
        n_step = n_step + 1
    padding = (n_step-1)*step_size + xy_window_size - original_image_size

    return n_step, padding

In [8]:
def register_imgStk(img_stack,convert_to_gray_to_reg=False):
    # from pystackreg import StackReg
    stitch_stack = None
    for img in img_stack:

        if convert_to_gray_to_reg:
            img = rgbStk_to_grayStk(img)
        else:
            img = img[:,:,0]

        img = np.expand_dims(img,0)

        if stitch_stack is None:
            stitch_stack = img
        else:
            stitch_stack = np.concatenate((stitch_stack,img),axis=0)

    sr = StackReg(StackReg.RIGID_BODY)
    tmats = sr.register_stack(stitch_stack, reference='previous')

    f_img = flattenClChannel(img_stack)
    out = np.zeros(f_img.shape).astype(float)

    expand_tmat_list = list(np.repeat([3],tmats.shape[0]))
    expand_tmat = np.repeat(tmats,expand_tmat_list,axis=0)

    for im_index in range(f_img.shape[0]):
        out[im_index, :, :] = transform.warp(f_img[im_index, :, :], expand_tmat[im_index, :, :], order=3)*255

    r_img = recoverClChannel(out).astype(np.uint8)

    return r_img

def select_infocus_layer_for_rgbImgStk(img_stack, best_focus):
    stack,h,w,c = img_stack.shape
    image = img_stack.reshape((stack,-1,c)) # image is now (stack, nr_pixels)

    best_focus = best_focus.ravel()

    r = np.array([image[best_focus[i],i] for i in range(image.shape[1])])
    r = r.reshape((h,w,c))

    return r

def rgbStk_to_grayStk(img_stack):
    gray_stack = None
    for i in range(img_stack.shape[0]):
        tmp = rgbImg_to_grayImg(img_stack[i])
        tmp = np.expand_dims(tmp,0)
        if gray_stack is None:
            gray_stack = tmp
        else:
            gray_stack = np.concatenate((gray_stack,tmp),axis=0)

    return gray_stack

def flattenClChannel(img_stack):
    z,x,y,c = img_stack.shape
    img_stack = np.moveaxis(img_stack,3,1)
    img_stack = img_stack.reshape((-1,x,y))
    return img_stack

def recoverClChannel(img_stack):
    z,x,y = img_stack.shape
    img_stack = img_stack.reshape((-1,3,x,y))
    img_stack = np.moveaxis(img_stack,1,3)
    return img_stack

def rgbImg_to_grayImg(rgb):
    return np.dot(rgb[...,:3], [0.2989, 0.5870, 0.1140])

In [9]:
def generate_edof_var_LayerMap_patch(img_stack,window_size):
    z,x,y,c = img_stack.shape
    patches = patchify(img_stack,(z,window_size,window_size,c),step=window_size)
    patches = np.squeeze(patches)
    patch_dim_0 = patches.shape[0]
    patch_dim_1 = patches.shape[1]
    patches = patches.reshape((-1,z,window_size,window_size,c))

    tmp = []
    for patch in patches:
        std_list = [np.var(patch[i]) for i in range(z)]
        index = std_list.index(max(std_list))
        tmp.append(index)

    tmp = np.array(tmp)
    # patch_dim = int(tmp.shape[0]**0.5)
    tmp = tmp.reshape(patch_dim_0,patch_dim_1)
    return tmp



def edof_var_patch(img_stack,window_size):
    z,x,y,c = img_stack.shape
    patches = patchify(img_stack,(z,window_size,window_size,c),step=window_size)
    patches = np.squeeze(patches)
    patch_dim_0 = patches.shape[0]
    patch_dim_1 = patches.shape[1]
    patches = patches.reshape((-1,z,window_size,window_size,c))

    tmp = []
    for patch in patches:
        std_list = [np.var(patch[i]) for i in range(z)]
        index = std_list.index(max(std_list))
        tmp.append(patch[index])

    tmp = np.array(tmp)
    #patch shape might not be square
    # patch_dim = int(tmp.shape[0]**0.5)
    tmp = tmp.reshape(patch_dim_0,patch_dim_1,window_size,window_size,c)
    tmp = np.expand_dims(tmp,axis=2)
    reconstructed_image = unpatchify(tmp, (x,y,c))

    return reconstructed_image

In [10]:
def scale_bounding_box(bbox, original_shape, original_scale, new_scale):
    """
    Scale bounding box coordinates based on the scale factors of two images.

    Args:
    bbox (tuple): The bounding box in the format (x, y, width, height).
    original_shape (tuple): The shape (height, width) of the first image.
    original_scale (float): The scale factor of the first image.
    new_scale (float): The scale factor of the second image.

    Returns:
    tuple: Scaled bounding box coordinates.
    """
    x, y, width, height = bbox
    original_height, original_width = original_shape

    # Calculate the effective scale factors
    scale_x = (original_width * new_scale) / (original_width * original_scale)
    scale_y = (original_height * new_scale) / (original_height * original_scale)

    # Scale the bounding box
    scaled_x = int(x * scale_x)
    scaled_y = int(y * scale_y)
    scaled_width = int(width * scale_x)
    scaled_height = int(height * scale_y)

    return (scaled_x, scaled_y, scaled_width, scaled_height)

# Example usage:
# bbox = (100, 150, 50, 75)  # Original bounding box (x, y, width, height)
# original_shape = (1000, 1000)  # Shape of the first image (height, width)
# original_scale = 1.0  # Scale factor of the first image
# new_scale = 0.5  # Scale factor of the second image
# scaled_bbox = scale_bounding_box(bbox, original_shape, original_scale, new_scale)
# print("Scaled Bounding Box:", scaled_bbox)

def bbox_trans_coord(bbox, new_coord_info):
    """
    Translate bounding box coordinates and check if it fits within a new coordinate system.

    Args:
    bbox (tuple): The bounding box in the format (x, y, width, height).
    new_origin (tuple): The new origin coordinates (new_x, new_y), which can be negative.
    new_system_width (int): The width of the new coordinate system.
    new_system_height (int): The height of the new coordinate system.

    Returns:
    tuple: Translated bounding box and a boolean indicating if it fits in the new system.
    """
    x, y, width, height = bbox
    new_x, new_y, new_width, new_height = new_coord_info

    # Translate the bounding box
    translated_x = x + new_x
    translated_y = y + new_y

    return (translated_x, translated_y, width, height)

# Example usage
# bbox = (100, 150, 50, 75)  # Original bounding box (x, y, width, height)
# new_origin = (-50, 50)  # New origin coordinates with negative x
# new_system_width, new_system_height = 200, 200  # Dimensions of the new coordinate system
# translated_bbox, fits = translate_and_check_bounding_box(bbox, new_origin, new_system_width, new_system_height)
# print("Translated Bounding Box:", translated_bbox, "Fits in new system:", fits)

def adjust_bbox_coord(n, bbox):
    bbox = list(bbox)
    bbox[0] = bbox[0] + n
    bbox[1] = bbox[1] + n
    bbox[2] = bbox[2] - 2*n
    bbox[3] = bbox[3] - 2*n
    bbox = tuple(bbox)
    return bbox

In [11]:
def get_full_coord_info(czi):
    full_coord_info = czi.get_mosaic_bounding_box()
    x,y,w,h = full_coord_info.x,full_coord_info.y,full_coord_info.w,full_coord_info.h
    return (x,y,w,h)

def get_contours_info_from_low_res(czi):
    msk = czi.read_mosaic(C=0, Z=0,scale_factor=0.1)[0,0]
    msk[msk!=0]=1
    contours, hierarchy = cv2.findContours(
        msk[:,:,0], cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    return msk, contours

def calc_contour_coord_in_full_res(contour, low_res_img_shape, new_coord_info):
    try:
        low_res_img_shape = low_res_img_shape.shape[:2]
    except:
        low_res_img_shape = low_res_img_shape
        
    bbox = cv2.boundingRect(contour)
    full_bbox = scale_bounding_box(bbox, low_res_img_shape, 0.1, 1)
    trans_full_bbox = bbox_trans_coord(full_bbox, new_coord_info)
    return trans_full_bbox

def validate_contour(contour, czi, max_range=200):
    orginal_contour = contour
    try:
        img = czi.read_mosaic(orginal_contour, C=0, Z=0,scale_factor=0.1)[0,0]
        return orginal_contour
    except:
        contour = orginal_contour
        for n in range(10,max_range,10):
            contour = adjust_bbox_coord(n, orginal_contour)
            try:
                img = czi.read_mosaic(contour, C=0, Z=0,scale_factor=0.1)[0,0]
                return contour
            except:
                print(f'Adjusting img size by shrink {n} pixels')
    raise Exception(f"Bounding box not fit in the original coordinate, try increase the n range, now is {max_range}") 

def obtain_img_zlayer_by_contour(contour, czi, z, scale_factor=0.1):
    img = czi.read_mosaic(contour, C=0, Z=z,scale_factor=scale_factor)[0,0]    
    img[:,:,[0,2]] = img[:,:,[2,0]] #BGR -> RGB
    return img    

def process_patch(img, window_shape, x_col, y_row, step, padding, x_pad, y_pad):
    z_start_point, c_start_point = 0, 0
    x_start_point, y_start_point = x_col * step, y_row * step
    start_points = [z_start_point, x_start_point, y_start_point, c_start_point]
    padding_values = (x_pad, y_pad) if padding else None

    return slice_func(img, start_points, window_shape, padding_values)

def apply_patch_processing(patch, filter_window_size, registration, smooth=None):
    if registration:
        reg_patch = register_imgStk(patch)
    else:
        reg_patch = patch

    if smooth:
        extend_dof_img_map = generate_edof_var_LayerMap_patch(reg_patch, filter_window_size)
        expand_map = cv2.resize(extend_dof_img_map.astype(float), (patch.shape[2], patch.shape[1]), interpolation=smooth)
        expand_map = expand_map.round().astype('int').clip(0, patch.shape[0] - 1)
        return select_infocus_layer_for_rgbImgStk(patch, expand_map)
    else:
        return edof_var_patch(reg_patch, filter_window_size) 


In [12]:
# def generate_output_path(output_name, output_path, file_path, suffix='flatten', img_format='jpg'):
#     head, tail = os.path.split(file_path)

#     if output_name is None:
#         print(f'output_name not set, will save with suffix {suffix}')
#         output_name = '{}_{}.{}'.format(tail.split('.')[0], suffix, img_format)

#     if output_path is None:
#         print('output_path not set, will save to the readin file path')
#         output_path = head

#     return os.path.join(output_path, output_name)

In [13]:
def estimate_file_size_from_contour(contour, czi, z_num):
    img = obtain_img_zlayer_by_contour(contour, czi, 0, 0.1)
    single_img_size = round(img.nbytes*100 / 1024 / 1024 /1024,2) #in Gb
    full_img_size = z_num * single_img_size #in Gb
    return single_img_size, full_img_size

In [14]:
def split_contour(contour_info, window_size):
    """
    Splits a contour into smaller pieces.
    
    :param x: X-coordinate of the contour's top-left corner.
    :param y: Y-coordinate of the contour's top-left corner.
    :param w: Width of the contour.
    :param h: Height of the contour.
    :param window_size: Size of the window for splitting.
    :return: A list of tuples (x, y, w, h) for each piece.
    """
    x,y,w,h = contour_info
    pieces = []
    for i in range(y, y + h, window_size):
        for j in range(x, x + w, window_size):
            # Adjust width and height for pieces at the edges
            width = min(window_size, x + w - j)
            height = min(window_size, y + h - i)
            pieces.append((j, i, width, height))
    return pieces

def fit_shape_with_window(shape, window_shape):
    w,h = shape
    res_w = w if w%window_shape == 0 else window_shape * (math.floor(w/window_shape)+1)
    res_h = h if h%window_shape == 0 else window_shape * (math.floor(h/window_shape)+1)
    return (res_w, res_h)

# Main Func

## From CZI to EDOF img directly patch

In [15]:
def read_czi_generate_edof_img_patch(file_path, 
                                output_name=None, output_path=None,
                                patch_window_factor=25, filter_window_size=200,
                                registration=False, smooth=False):

    readin_patch_size = patch_window_factor*filter_window_size
    scale_factor=1
    
    print(f'File:{file_path} Processing')
    czi = CziFile(file_path)
    z_range = czi.get_dims_shape()[0]['Z'][1]
    full_coord_info = get_full_coord_info(czi)

    msk, low_res_contours = get_contours_info_from_low_res(czi)
    output_img_num = len(low_res_contours)
    channel_num = msk.shape[-1]

    for c_idx, low_res_contour in enumerate(low_res_contours, 1):
        print(f'Multi Z Layer Img Reading & EDOF {c_idx}/{output_img_num}')
        full_res_contour_bbox = calc_contour_coord_in_full_res(low_res_contour, msk, full_coord_info)
        full_res_contour_bbox = validate_contour(full_res_contour_bbox, czi)
    
        scale_shape = [int(1.01*scale_factor*i) for i in full_res_contour_bbox[2:][::-1]]
        output_wh = fit_shape_with_window(scale_shape, readin_patch_size)
        flatten_image = np.zeros((*output_wh, channel_num)).astype('uint8')
    
        normal_full_res_contour_bbox = (0,0,*full_res_contour_bbox[-2:])
        sub_contours = split_contour(normal_full_res_contour_bbox, readin_patch_size)
    
        for sub_contour in tqdm(sub_contours):
            trans_sub_contour = bbox_trans_coord(sub_contour, full_res_contour_bbox)
            img = obtain_img_zlayer_by_contour(trans_sub_contour, czi, 0, 0.1)
    
            if np.any(img):
                stack = []
                for z in range(z_range):
                    trans_sub_contour = bbox_trans_coord(sub_contour, full_res_contour_bbox)
                    img = obtain_img_zlayer_by_contour(trans_sub_contour, czi, z, scale_factor)
                    img = np.expand_dims(img, axis=0)
                    stack.append(img)
        
                stack = np.concatenate(stack, axis=0)
        
        
                stack_z, stack_w, stack_h, stack_c= stack.shape
                if stack_w < readin_patch_size or stack_h < readin_patch_size:
                    holder = np.zeros((stack_z, readin_patch_size, readin_patch_size, stack_c))
                    holder[:, :stack_w, :stack_h ,:] = stack
                    stack = holder
                    
                flatten_patch = apply_patch_processing(stack, filter_window_size, registration, smooth).astype('uint8')
        
                flatten_image[
                        sub_contour[1]:sub_contour[1]+readin_patch_size, 
                        sub_contour[0]:sub_contour[0]+readin_patch_size,
                        :
                ] = flatten_patch
                
                del flatten_patch, stack; gc.collect()
            else:
                del img; gc.collect()
    
        final_image_path = generate_output_path(output_name, output_path, file_path, c_idx, suffix='WholeImage', img_format='jpg')
    
        print('Croping...', end='')
        out_w, out_h = full_res_contour_bbox[-2:]
        flatten_image = flatten_image[:out_h+20, :out_w+20]
        print('Saving...', end='')
        writeTiff(flatten_image, final_image_path) 
        print('Done')
            
        del flatten_image; gc.collect()
    print('---------------------------')
    return 0

## From CZI to EDOF img directly Full

In [16]:
def generate_output_path(output_name, output_path, file_path, img_num, suffix='flatten', img_format='jpg'):
    head, tail = os.path.split(file_path)

    if output_name is None:
        print(f'output_name not set, will save with suffix {suffix}')
        originl_file_name = tail.split('.')[0]
        output_name = f'{originl_file_name}_S{img_num}_{suffix}.{img_format}'
    else:
        output_name = f'{output_name}_S{img_num}_{suffix}.{img_format}'

    if output_path is None:
        print('output_path not set, will save to the readin file path')
        output_path = head


    return os.path.join(output_path, output_name)
    
def read_czi_generate_edof_img(file_path, 
                                output_name=None, output_path=None,
                                z_range=None, scale_factor=0.1,
                                patch_window_size=2000,filter_window_size=200,
                                registration=False, smooth=False):


    padding = True
    
    print(f'File:{file_path} Processing')
    czi = CziFile(file_path)

    if z_range == None:
        z_range = czi.get_dims_shape()[0]['Z'][1]
    
    full_coord_info = get_full_coord_info(czi)
    msk, low_res_contours = get_contours_info_from_low_res(czi)
    output_img_num = len(low_res_contours)
    # print(f'{output_img_num} output image(s)')

    for c_idx, low_res_contour in enumerate(low_res_contours, 1):
        print(f'Multi Z Layer Img {c_idx}/{output_img_num}')
        high_res_contour_bbox = calc_contour_coord_in_full_res(low_res_contour, msk, full_coord_info)
        high_res_contour_bbox = validate_contour(high_res_contour_bbox, czi)

        #estimate full image size here      
        start_time = time.time()
        stack = []
        for z in range(z_range):
            img = obtain_img_zlayer_by_contour(high_res_contour_bbox, czi, z, scale_factor)
            img = np.expand_dims(img, axis=0)
            stack.append(img)
            
            worked_time = (time.time()-start_time)/60
            progress = 100*(z+1)/z_range
            time_left = worked_time/(z+1) * (z_range-(z+1))
            print(f'\rReading:\t Progress:{progress:.2f}% | Run:{worked_time:.2f} min | est Time left:{time_left:.2f} min | {z+1}/{z_range}...', end = '')
        print('Finished')

        print('\t\tConcatenating...', end='')
        stack = np.concatenate(stack, axis=0)
        print('Finished')
        
        del img
        gc.collect()

        step = xy_window_size = patch_window_size
        z, x, y, c = stack.shape
        window_shape = [z, xy_window_size, xy_window_size, c]
        x_cols, x_pad = calc_pad_parms(x, xy_window_size, step)
        y_rows, y_pad = calc_pad_parms(y, xy_window_size, step)
        
        whole_image = np.zeros((x_cols * xy_window_size, y_rows * xy_window_size, 3), dtype='uint8')
        total_count = y_rows * x_cols

        start_time = time.time()
        worked_time = 0 
        time_left = 0
        for count, (y_row, x_col) in enumerate(itertools.product(range(y_rows), range(x_cols)), 1):
            img_slice = process_patch(stack, window_shape, x_col, y_row, step, padding, x_pad, y_pad)
            processed_patch = apply_patch_processing(img_slice, filter_window_size, registration, smooth)
            
            whole_image[x_col * xy_window_size: (x_col + 1) * xy_window_size, 
                y_row * xy_window_size: (y_row + 1) * xy_window_size] = processed_patch
            
            worked_time = (time.time()-start_time)/60
            progress = 100*count/total_count
            time_left = worked_time/count * (total_count-count)
            print(f'\rEDOF:\t\t Progress:{progress:.2f}% | Run:{worked_time:.2f} min | est Time left:{time_left:.2f} min | {count}/{total_count}...', end='')
        print('Finished')
        
        del stack
        gc.collect()
        
        
        final_image_path = generate_output_path(output_name, output_path, file_path, c_idx, suffix='WholeImage', img_format='jpg')
        
        print('\t\tCroping...', end='')
        whole_image = crop_content(whole_image)
        
        print('Saving...', end='')
        writeTiff(whole_image, final_image_path) 
        print('Done')
        
        del whole_image
        gc.collect()

    print('---------------------------')
    return 0 

## Generate Scale multi zLayer img fron CZI

In [17]:
def read_czi_generate_scale_img(file_path, z_range=None, output_name=None, output_path=None,
                             scale_factor=0.1):

    output_file_path = setup_output_paths(output_name, output_path)
    print(f'File:{file_path} Processing')

    czi = CziFile(file_path)

    if z_range == None:
        z_range = czi.get_dims_shape()[0]['Z'][1]
        
    full_coord_info = get_full_coord_info(czi)
    msk, low_res_contours = get_contours_info_from_low_res(czi)
    output_img_num = len(low_res_contours)
    print(f'{output_img_num} output image(s)')
    
    start_time = time.time()
    worked_time = 0 
    time_left = 0
    count = 1
    total_count = output_img_num*z_range

    for c_idx, low_res_contour in enumerate(low_res_contours):
        high_res_contour_bbox = calc_contour_coord_in_full_res(low_res_contour, msk, full_coord_info)
        high_res_contour_bbox = validate_contour(high_res_contour_bbox, czi)

        #estimate full image size here
        tif = tifffile.TiffWriter('{}_S{}.tif'.format(output_file_path, c_idx+1), bigtiff=True)
        for z in range(z_range):
            img = obtain_img_zlayer_by_contour(high_res_contour_bbox, czi, z, scale_factor)        
            tif.write(img, contiguous=True)
            
            worked_time = (time.time()-start_time)/60
            progress = 100*count/total_count
            time_left = worked_time/count * (total_count-count)
            print(f'\r{output_name}  {count}/{total_count} Progress:{progress:.2f}% | Run:{worked_time:.2f} min | est Time left:{time_left:.2f} min',end='')
            count += 1
            
        tif.close()
        
    print('\n------Done------')
    return output_img_num

def setup_output_paths(output_name, output_path):
    if output_path is None and output_name is None:
        print('Warning: output_path is not set, output file could be rewritten!')

    if output_name is None:
        output_name = 'output_img'

    if output_path is None:
        output_path = ''

    return os.path.join(output_path, output_name)

## EDOF the multi zLayer Img

In [18]:
def edof_multi_zLayer_img(img, xy_window_size, save_file_name='output_img',
                            save_file_path='', filter_window_size=20, registration=False,
                            smooth=None):

    # smooth_method=cv2.INTER_CUBIC,

    """
    Process an image in patches and directly save each patch or the whole processed image.

    Args:
        img: Image to be processed.
        xy_window_size: Size of the window for patching.
        step: Step size for patching.
        save_file_name: Base name for saved files.
        save_file_path: Directory where files will be saved.
        generate_image: Flag to generate individual patch images.
        generate_whole_image: Flag to generate a whole image from patches.
        padding: Apply padding to patches.
        filter_window_size: Window size for filtering in patch processing.
        registration: Apply registration to patches.
        smooth: Apply smoothing to patches.
        smooth_method: Smoothing method to be used.
        save_format: Format for saving the images.

    Returns:
        Tuple of number of rows and columns in the processed image.
    """
    
    step = xy_window_size
    padding = True
    z, x, y, c = img.shape
    window_shape = [z, xy_window_size, xy_window_size, c]
    x_cols, x_pad = calc_pad_parms(x, xy_window_size, step)
    y_rows, y_pad = calc_pad_parms(y, xy_window_size, step)

    whole_image = np.zeros((x_cols * xy_window_size, y_rows * xy_window_size, 3), dtype='uint8')

    patch_num = y_rows * x_cols
    start_time = time.time()

    for count, (y_row, x_col) in enumerate(itertools.product(range(y_rows), range(x_cols)), 1):
        img_slice = process_patch(img, window_shape, x_col, y_row, step, padding, x_pad, y_pad)
        processed_patch = apply_patch_processing(img_slice, filter_window_size, registration, smooth)
        
        whole_image[x_col * xy_window_size: (x_col + 1) * xy_window_size, 
                y_row * xy_window_size: (y_row + 1) * xy_window_size] = processed_patch

        report_progress(count, patch_num, start_time, save_file_name)

    del img
    gc.collect()
    
    final_image_path = construct_image_path(save_file_name, save_file_path, 'WholeImage')
    print('\nSaving image')
    whole_image = crop_content(whole_image)
    writeTiff(whole_image, final_image_path)

    print('-----Done-----\n')
    return y_rows, x_cols
    
def report_progress(current_count, total_count, start_time, save_file_name):
    worked_time = (time.time() - start_time) / 60
    progress = 100 * current_count / total_count
    time_left = worked_time / current_count * (total_count - current_count)
    print(f'\r{save_file_name}  {current_count}/{total_count} Progress: {progress:.2f}% | Run: {worked_time:.2f} min | Est. Time Left: {time_left:.2f} min', end='')
    
def construct_image_path(save_file_name, save_file_path, suffix):
    return os.path.join(save_file_path, f'{save_file_name}_{suffix}.jpg')

# Single file work flow

## One step (czi -> edof img)

In [22]:
folder_path = r'C:\Users\zhong\Documents\Python_Project\EDOF'
file_path = r'2023_11_20__12_58__0540.czi'
file_path = os.path.join(folder_path, file_path)
output_name = '2023_11_20__12_58__0540' 
output_path = r''

scale_factor = 1
z_range = 11
patch_window_size = 4000
filter_window_size = 200
smooth = None
registration = False

# '''
# smooth = 
# cv2.INTER_AREA,
# cv2.INTER_BITS,
# cv2.INTER_BITS2,
# cv2.INTER_CUBIC,
# cv2.INTER_LANCZOS4,
# cv2.INTER_LINEAR,
# cv2.INTER_LINEAR_EXACT,
# cv2.INTER_NEAREST

# https://stackoverflow.com/questions/23853632/which-kind-of-interpolation-best-for-resizing-image
# '''



### split patch reading

In [ ]:
read_czi_generate_edof_img_patch(file_path, 
                                output_name=output_name, output_path=output_path,
                                patch_window_factor=25, filter_window_size=200,
                                registration=False, smooth=False)

### full layer reading

In [22]:
read_czi_generate_edof_img(
                            file_path, 
                            output_name=output_name, 
                            output_path=output_path,
                            z_range=z_range,
                            scale_factor=scale_factor,
                            patch_window_size=patch_window_size,
                            filter_window_size=200,
                            registration=False, 
                            smooth=smooth
)

File:E:\20231120_ks NSRL-22A\2023_11_20__13_03__0564.czi Processing
Multi Z Layer Img 1/2
Reading:	 Progress:100.00% 	| Run:6.61 min 	| est Time left:0.00 min 	| 11/11...Finished
Concatenating...Finished
EDOF:	 Progress:100.00% 	| Run:1.96 min 	| est Time left:0.00 min 	| 64/64...Finished
Croping...Saving...Done
Multi Z Layer Img 2/2
Reading:	 Progress:100.00% 	| Run:8.00 min 	| est Time left:0.00 min 	| 11/11...Finished
Concatenating...Finished
EDOF:	 Progress:100.00% 	| Run:1.73 min 	| est Time left:0.00 min 	| 60/60...Finished
Croping...Saving...Done
---------------------------


0

## Two steps (czi -> multi z layers img -> edof img)

In [17]:
folder_path = r'E:\20231120_ks NSRL-22A'
file_path = r'2023_11_20__13_03__0565.czi'
file_path = os.path.join(folder_path, file_path)
output_name = '2023_11_20__13_03__0565' 
output_path = r'E:\20231120_ks NSRL-22A\flatten'

scale_factor = 1
z_range = 11

In [18]:
img_num = read_czi_generate_scale_img(
                                        file_path, 
                                        z_range, 
                                        output_name=output_name, 
                                        output_path=output_path,
                                        scale_factor=scale_factor
)

File:E:\20231120_ks NSRL-22A\2023_11_20__13_03__0565.czi Processing
2 output image(s)
2023_11_20__13_03__0565  22/22 Progress:100.00% | Run:6.98 min | est Time left:0.00 min
------Done------


In [20]:
patch_window_size = 4000
filter_window_size = 200
smooth = None
# '''
# smooth = 
# cv2.INTER_AREA,
# cv2.INTER_BITS,
# cv2.INTER_BITS2,
# cv2.INTER_CUBIC,
# cv2.INTER_LANCZOS4,
# cv2.INTER_LINEAR,
# cv2.INTER_LINEAR_EXACT,
# cv2.INTER_NEAREST

# https://stackoverflow.com/questions/23853632/which-kind-of-interpolation-best-for-resizing-image
# '''

In [25]:
for num in range(img_num):
    num += 1
    img_name = '{}_S{}.tif'.format(output_name,num)
    img_path = os.path.join(output_path,img_name)
    print('Loading the file...',end='')
    img = readTiffStk(img_path,move_axis=False)
    print('Finish')

    save_file_name = '{}_S{}'.format(output_name,num)
    save_file_path = output_path

    y_rows,x_cols = edof_multi_zLayer_img(
                                            img,
                                            patch_window_size,
                                            save_file_name,
                                            save_file_path,
                                            filter_window_size=filter_window_size,
                                            registration=False,
                                            smooth=smooth
    )

    del img
    gc.collect()
    os.remove(img_path)

Loading the file...Finish
2023_11_20__13_03__0565_S1  20/20 Progress: 100.00% | Run: 0.59 min | Est. Time Left: 0.00 min
Saving image
-----Done-----

Loading the file...Finish
2023_11_20__13_03__0565_S2  20/20 Progress: 100.00% | Run: 0.58 min | Est. Time Left: 0.00 min
Saving image
-----Done-----



# Folder workflow

## Two steps

In [ ]:
folder_path = r'E:\20231120_ks NSRL-22A'
folder_list = os.listdir(folder_path)
folder_list = [i for i in folder_list if i.split('.')[-1]=='czi']

output_path = r'E:\20231120_ks NSRL-22A\flatten'

z_range = 11
scale_factor = 1

patch_window_size = 4000
patch_window_step = 4000
filter_window_size = 200
smooth = False
smooth_method = cv2.INTER_CUBIC

In [ ]:
for file_name in folder_list:
    full_file_path = os.path.join(folder_path,file_name)
    output_name = file_name.split('.')[0]
    
    img_num = read_scale_save_czi2tiff(full_file_path, 
                                       z_range, 
                                       output_name=output_name, 
                                       output_path=output_path,
                                       scale_factor=scale_factor)
    
    for num in range(img_num):
        num += 1
        img_name = '{}_S{}.tif'.format(output_name,num)
        img_path = os.path.join(output_path,img_name)
        img = readTiffStk(img_path,move_axis=False)

        save_file_name = '{}_S{}'.format(output_name,num)
        save_file_path = output_path

        y_rows,x_cols = patchiProcessDirectSave(
                                            img,
                                            patch_window_size,
                                            patch_window_step,
                                            save_file_name,
                                            save_file_path,
                                            generate_image=False,
                                            generate_whole_image=True,
                                            padding=True,
                                            filter_window_size=filter_window_size, 
                                            smooth=smooth, 
                                            smooth_method=smooth_method, 
                                            save_format='jpg')

        del img
        gc.collect()
        os.remove(img_path)

## One step

### Full layer

In [33]:
folder_path = r'E:\20231124_ks_RR10_BNL2_BNL3'
folder_list = os.listdir(folder_path)
folder_list = [i for i in folder_list if i.split('.')[-1]=='czi']

output_path = r'E:\20231124_ks_RR10_BNL2_BNL3\flatten'

z_range = 11
scale_factor = 1

patch_window_size = 4000
filter_window_size = 200
smooth = False
registration = False
# smooth_method = cv2.INTER_CUBIC

In [34]:
for file_name in folder_list:
    full_file_path = os.path.join(folder_path,file_name)
    output_name = file_name.split('.')[0]
        
    read_czi_generate_edof_img(
                                full_file_path, 
                                output_name=output_name, 
                                output_path=output_path,
                                z_range=z_range,
                                scale_factor=scale_factor,
                                patch_window_size=patch_window_size,
                                filter_window_size=filter_window_size,
                                registration=registration, 
                                smooth=smooth
    )

File:E:\20231124_ks_RR10_BNL2_BNL3\2023_11_24__12_28__0674.czi Processing
Multi Z Layer Img 1/2
Reading:	 Progress:100.00% | Run:2.99 min | est Time left:0.00 min | 11/11...Finished
		Concatenating...Finished
EDOF:		 Progress:100.00% | Run:1.05 min | est Time left:0.00 min | 36/36...Finished
		Croping...Saving...Done
Multi Z Layer Img 2/2
Adjusting img size by shrink 10 pixels
Reading:	 Progress:100.00% | Run:2.97 min | est Time left:0.00 min | 11/11...Finished
		Concatenating...Finished
EDOF:		 Progress:100.00% | Run:0.81 min | est Time left:0.00 min | 28/28...Finished
		Croping...Saving...Done
---------------------------
File:E:\20231124_ks_RR10_BNL2_BNL3\2023_11_24__12_28__0675.czi Processing
Multi Z Layer Img 1/1
Reading:	 Progress:100.00% | Run:5.60 min | est Time left:0.00 min | 11/11...Finished
		Concatenating...Finished
EDOF:		 Progress:100.00% | Run:1.23 min | est Time left:0.00 min | 42/42...Finished
		Croping...Saving...Done
---------------------------
File:E:\20231124_ks_RR

RuntimeError: This file is not a mosaic file but was assumed to be. Use the non-mosaic specific bounding box functions.

### patch

In [19]:
folder_path = r'D:\DATASET\brightfield_wsi_flatten_imgs\BNL3\SR'
folder_list = os.listdir(folder_path)
folder_list = [i for i in folder_list if i.split('.')[-1]=='czi']

output_path = r'D:\DATASET\brightfield_wsi_flatten_imgs\BNL3\SR\flatten'

filter_window_size = 200
smooth = False #cv2.INTER_CUBIC
registration = False
file_num = len(folder_list)

for idx,file_name in enumerate(folder_list,1):
    full_file_path = os.path.join(folder_path,file_name)
    output_name = file_name.split('.')[0]
    
    print(f'File: {idx}/{file_num}')
    read_czi_generate_edof_img_patch(full_file_path, 
                                    output_name=output_name, output_path=output_path,
                                    patch_window_factor=25, filter_window_size=filter_window_size,
                                    registration=False, smooth=False)

File: 1/44
File:D:\DATASET\brightfield_wsi_flatten_imgs\BNL3\SR\2023_04_04__16_53__0544.czi Processing
Multi Z Layer Img Reading & EDOF 1/1


100%|██████████████████████████████████████████████████████████████████████████████████| 25/25 [13:42<00:00, 32.90s/it]


Croping...Saving...Done
---------------------------
File: 2/44
File:D:\DATASET\brightfield_wsi_flatten_imgs\BNL3\SR\2023_04_04__16_53__0545.czi Processing
Multi Z Layer Img Reading & EDOF 1/1


100%|██████████████████████████████████████████████████████████████████████████████████| 25/25 [13:21<00:00, 32.04s/it]


Croping...Saving...Done
---------------------------
File: 3/44
File:D:\DATASET\brightfield_wsi_flatten_imgs\BNL3\SR\2023_04_04__16_53__0546.czi Processing
Multi Z Layer Img Reading & EDOF 1/1


...

Croping...Saving...Done
---------------------------
File: 43/44
File:D:\DATASET\brightfield_wsi_flatten_imgs\BNL3\SR\2023_04_04__17_02__0586.czi Processing
Multi Z Layer Img Reading & EDOF 1/1


100%|██████████████████████████████████████████████████████████████████████████████████| 25/25 [14:11<00:00, 34.07s/it]


Croping...Saving...Done
---------------------------
File: 44/44
File:D:\DATASET\brightfield_wsi_flatten_imgs\BNL3\SR\2023_04_04__17_02__0587.czi Processing
Multi Z Layer Img Reading & EDOF 1/1


100%|██████████████████████████████████████████████████████████████████████████████████| 25/25 [14:38<00:00, 35.15s/it]


Croping...Saving...Done
---------------------------
